In [1]:
import pyJHTDB
import pyJHTDB.dbinfo
import numpy as np
import matplotlib.pyplot as plt
import time

### Data for 4.3
We consider a larger domain covering half channel height in this test. We
consider a subdomain at [12.47, 12.66] × [-1, -0.0031] × [4.61, 4.82] (about
190 × 997 × 210 in wall-units) as the VP-NSFnet simulation domain; and the
non-dimensional time domain is set as [0, 0.104] (17 time steps, 5.19 in wallunits). We place 100,000 points inside the domain and 26,048 points on the
boundary sampled at each time step, and 147,968 points at the initial time step
to determine the loss function. 

In [2]:
def get_data(point_coords, time):
    
    """
    Get velocity and pressure at specified spatial points and a specified time in channel flow database.
    :param point_coords: Spatial coordinates of the data points of interest. Must be in single precision.
    :param time: Time of interest.
    :return: Velocity and velocity gradient arrays.
    """

    # Create library object
    lJHTDB = pyJHTDB.libJHTDB()

    # Initialize library object
    lJHTDB.initialize()

    # Get velocity
    u = lJHTDB.getData(time, point_coords,
                       sinterp='Lag4',
                       data_set='channel',
                       getFunction='getVelocity')

    # Get velocity gradient
    p = lJHTDB.getData(time, point_coords,
                            sinterp='Lag4',
                            data_set='channel',
                            getFunction='getPressure')

    # Finalize library object
    lJHTDB.finalize()

    return u, p

### Data for initial condition

In [3]:
xnode = np.linspace(12.47,12.66,191)
ynode = np.linspace(-1,-0.0031,998)
znode = np.linspace(4.61,4.82,211)
total_times = np.array(list(range(17)), dtype = np.float32) * 0.0065

points = np.zeros((191, 998, 211, 3), np.float32)

points[:, :, :, 0] = xnode[:, None, None]
points[:, :, :, 1] = ynode[None, :, None]
points[:, :, :, 2] = znode[None, None, :]


In [4]:
points1 = points.reshape(-1,3)
points1.shape

(40220398, 3)

In [6]:
ini_number = 147968 
idx_ini = np.random.choice(points1.shape[0], ini_number, replace = False) 
train_ini1 = points1[idx_ini, :] 
train_iniv1 = np.zeros((ini_number, 3), np.float32) 
train_inip1 = np.zeros((ini_number, 1), np.float32) 

In [8]:
# 147968 points at the initial time step
size = int(ini_number / 68)
for i in range(68):
    
    start = time.time()  # start timer

    train_iniv1[0 + size * i: size + size * i, :], train_inip1[0 + size * i: size + size * i, :] = get_data(train_ini1[0 + size * i: size + size * i, :], 0)

    if i % 10 == 0:
        print('Elapsed time %d = %.2f seconds' % (i, elapsed))
        elapsed = time.time() - start  # end timer

print(train_iniv1.shape, train_inip1.shape)

Elapsed time 0 = 0.46 seconds
Elapsed time 1 = 0.42 seconds
Elapsed time 2 = 0.45 seconds
Elapsed time 3 = 0.43 seconds
Elapsed time 4 = 0.45 seconds
Elapsed time 5 = 0.51 seconds
Elapsed time 6 = 0.46 seconds
Elapsed time 7 = 0.45 seconds
Elapsed time 8 = 0.42 seconds
Elapsed time 9 = 0.47 seconds
Elapsed time 10 = 0.42 seconds
Elapsed time 11 = 0.44 seconds
Elapsed time 12 = 0.44 seconds
Elapsed time 13 = 0.42 seconds
Elapsed time 14 = 0.46 seconds
Elapsed time 15 = 0.49 seconds
Elapsed time 16 = 0.54 seconds
Elapsed time 17 = 0.59 seconds
Elapsed time 18 = 0.71 seconds
Elapsed time 19 = 0.43 seconds
Elapsed time 20 = 0.43 seconds
Elapsed time 21 = 0.45 seconds
Elapsed time 22 = 0.51 seconds
Elapsed time 23 = 0.42 seconds
Elapsed time 24 = 1.28 seconds
Elapsed time 25 = 0.47 seconds
Elapsed time 26 = 0.42 seconds
Elapsed time 27 = 0.44 seconds
Elapsed time 28 = 0.56 seconds
Elapsed time 29 = 0.46 seconds
Elapsed time 30 = 0.44 seconds
Elapsed time 31 = 0.43 seconds
Elapsed time 32 = 

In [9]:
# save the initial data as the npy
np.save('train_ini2.npy',train_ini1)
np.save('train_iniv2.npy',train_iniv1)
np.save('train_inip2.npy',train_inip1)

### Data for the boundary condition

In [10]:
points2 = points1[:, :][points1[:,0] == 12.47]
points3 = points1[:, :][points1[:,0] == 12.66]
points4 = points1[:, :][points1[:,1] == -1.0]
points5 = points1[:, :][points1[:,1] == -0.0031]
points6 = points1[:, :][points1[:,2] == 4.61]
points7 = points1[:, :][points1[:,2] == 4.82]
print(points2.shape, points3.shape, points4.shape, points5.shape, points6.shape, points7.shape)

(210578, 3) (210578, 3) (40301, 3) (40301, 3) (190618, 3) (190618, 3)


In [15]:
train_b1 = np.concatenate([points2, points3, points4, points5, points6, points7],0)
train_b1.shape

(882994, 3)

In [16]:
# 26048 points on the boundary
b_num = 26048
idxb = np.random.choice(train_b1.shape[0], b_num, replace=False)
train_b2 = train_b1[idxb,:]
train_b2.shape

(26048, 3)

In [17]:
train_xb1 = np.zeros((b_num*17, 4), np.float32)
train_vb1 = np.zeros((b_num*17, 3), np.float32)
train_pb1 = np.zeros((b_num*17, 1), np.float32)

In [19]:
frames = np.arange(17)
size = int(b_num / 8)
# Get data
for frame in frames:
    start = time.time()  # start timer
    t = total_times[frame]
    print('t = %s' % t)
    for i in range(8):
        train_vb1[b_num*frame + size*i: size+b_num*frame+size*i, :], train_pb1[b_num*frame + size*i: size+b_num*frame+size*i, :] = get_data(train_b2[size*i: size+size*i, :], t)
    train_xb1[b_num*frame: b_num+b_num*frame, 0:3] = train_b2
    train_xb1[b_num*frame: b_num+b_num*frame, 3] = t


t = 0.0
t = 0.0065
t = 0.013
t = 0.0195
t = 0.026
t = 0.0325
t = 0.039
t = 0.045500003
t = 0.052
t = 0.0585
t = 0.065
t = 0.0715
t = 0.078
t = 0.0845
t = 0.091000006
t = 0.097500004
t = 0.104


In [20]:
print(train_vb1.shape,train_pb1.shape,train_xb1.shape)
np.save('train_xb2.npy',train_xb1)
np.save('train_vb2.npy',train_vb1)
np.save('train_pb2.npy',train_pb1)

(442816, 3) (442816, 1) (442816, 4)


In [62]:
x0_train = train_ini1[:,0:1]
y0_train = train_ini1[:,1:2]
z0_train = train_ini1[:,2:3]
t0_train = np.zeros(train_ini1[:,0:1].shape, np.float32)
u0_train = train_iniv1[:,0:1]
v0_train = train_iniv1[:,1:2]
w0_train = train_iniv1[:,2:3]

xb_train = train_xb1[:,0:1]
yb_train = train_xb1[:,1:2]
zb_train = train_xb1[:,2:3]
tb_train = train_xb1[:,3:4]
ub_train = train_vb1[:,0:1]
vb_train = train_vb1[:,1:2]
wb_train = train_vb1[:,2:3]

x_train1 = xnode.reshape(-1,1)[np.random.choice(191, 100000, replace=True),:]
y_train1 = ynode.reshape(-1,1)[np.random.choice(998, 100000, replace=True),:]
z_train1 = znode.reshape(-1,1)[np.random.choice(211, 100000, replace=True),:] 
x_train = np.tile(x_train1,(17, 1)) 
y_train = np.tile(y_train1,(17, 1)) 
z_train = np.tile(z_train1,(17, 1)) 

total_times1 = np.array(list(range(17))) * 0.0065 
t_train1 = total_times1.repeat(100000) 
t_train = t_train1.reshape(-1,1) 

In [ ]:
# Database Spatial Interpolation Options:
# Interpolation options for GetVelocity, GetMagneticField, GetVectorPotential, GetPressure, GetVelocityAndPressure, GetDensity and GetPosition

# NoSInt: No Space interpolation (value at the datapoint closest to each coordinate value)
# Lag4: 4th-order Lagrange Polynomial interpolation along each spatial direction|
# Lag6: 6th-order Lagrange Polynomial interpolation along each spatial direction
# Lag8: 8th-order Lagrange Polynomial interpolation along each spatial direction
# M1Q4: Splines with smoothness 1 (3rd order) over 4 data points.
# M2Q8: Splines with smoothness 2 (5th order) over 8 data points.
# M2Q14: Splines with smoothness 2 (5th order) over 14 data points.

In [65]:
print(x0_train.shape,y0_train.shape,
      z0_train.shape,t0_train.shape,
      u0_train.shape,v0_train.shape,
      w0_train.shape,
      xb_train.shape,yb_train.shape,
      zb_train.shape,tb_train.shape,
      ub_train.shape,vb_train.shape,
      wb_train.shape,
      x_train.shape, y_train.shape,
      z_train.shape, t_train.shape)

(147968, 1) (147968, 1) (147968, 1) (147968, 1) (147968, 1) (147968, 1) (147968, 1) (442816, 1) (442816, 1) (442816, 1) (442816, 1) (442816, 1) (442816, 1) (442816, 1) (1700000, 1) (1700000, 1) (1700000, 1) (1700000, 1)


### Data for testing the result of NN

In [5]:
idx_test = np.random.choice(points1.shape[0],3000,replace = False)
test_l = points1[idx_test, :] # test points locations
#choose several different time step

test_v1, test_p1 = get_data(test_l, 0.0065)
test_v2, test_p2 = get_data(test_l, 4 * 0.0065)
test_v3, test_p3 = get_data(test_l, 7 * 0.0065)
test_v4, test_p4 = get_data(test_l, 10 * 0.0065)
test_v5, test_p5 = get_data(test_l, 13 * 0.0065)

test_v = np.concatenate((test_v1, test_v2, test_v3, test_v4, test_v5), 0)
test_p = np.concatenate((test_p1, test_p2, test_p3, test_p4, test_p5), 0)

test43 = np.concatenate((test_v, test_p), 1)

np.save('test43_l.npy', test_l)
np.save('test43_vp.npy', test43)

(15000, 4)